# # Wrangle OpenStreetMap Data

The goal of this project was to  use data munging techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap data for Austin, TX. 

## Objectives

* Assess the quality of the data for validity, accuracy, completeness, consistency and uniformity.
* Parsing and gather data from popular file formats such as .json, .xml, .csv, .html.
* Process data from many files and very large files that can be cleaned with spreadsheet programs.
* Learn how to store, query, and aggregate data using SQL.


## Project Introduction

The objective of this project is to take a data set from [OpenStreetMap] of Austin city to audit, clean, convert data from XML to CSV, and add it to a MySQL database.

In [118]:
## Auditing the Tag Types

#In order to audit the data I had to find out which type of tags existed within the file.

In [11]:
import xml.etree.cElementTree as ET
import pprint

osm_file = open("austin.osm", "r")

def count_tags(file):
	""" 
		Parses through tags and adds tag to dictionary 
		after calling the add_tag function 
		Args:
			file: .osm file containing the OpenStreetMap data
		Returns:
			tag_names: Dictionary containing tag type and count
	"""
	tag_names = {}
	for event, elem in ET.iterparse(file, events=("start",)):
		add_tag(elem.tag, tag_names)
	return tag_names

def add_tag(tag, tag_names):
	""" 
		Either adds tag to dictionary or adds a count to existing tag 
		Args:
			tag: Element tag for element in file
			tag_names: Dictionary of tag names and count
	"""
	if tag not in tag_names:
		tag_names[tag] = 1
	else:
		tag_names[tag] += 1

tags = count_tags(osm_file)
pprint.pprint(tags)

{'bounds': 1,
 'member': 16096,
 'meta': 1,
 'nd': 377627,
 'node': 340453,
 'note': 1,
 'osm': 1,
 'relation': 638,
 'tag': 181918,
 'way': 39979}


In [13]:

import xml.etree.cElementTree as ET
import pprint
import re

from collections import defaultdict

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

OSMFILE = "austin.osm"

def key_type(element, keys):
    if element.tag == "tag":
        for tag in element.iter('tag'):
            k = tag.get('k')
            if lower.search(element.attrib['k']):
                keys['lower'] = keys['lower'] + 1
            elif lower_colon.search(element.attrib['k']):
                keys['lower_colon'] = keys['lower_colon'] + 1
            elif problemchars.search(element.attrib['k']):
                keys['problemchars'] = keys['problemchars'] + 1
            else:
                keys['other'] = keys['other'] + 1
    
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

pprint.pprint(process_map(OSMFILE))

{'lower': 94621, 'lower_colon': 86372, 'other': 925, 'problemchars': 0}


In [ ]:
#For the purposes of this project, I will be looking at the node and way tags of this data set.
#Nodes are defined as a single point in space and is defined by longitude, latitude, and node id. 
#Ways are an ordered list of nodes that either define a region, closed node, or some linear feature, open node.


In [120]:
## Auditing the Users

#I used the below code
#to find out how many different contributors there have been to this data set.

In [25]:
import xml.etree.cElementTree as ET

def get_user(element):
    if 'uid' in element.attrib:
        user = element.get('uid')
        return user

def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        user = get_user(element)
        if user != None:
            users.add(user)

    return users

users = process_map('austin.osm')

print(len(users))

477


In [33]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

osm_file = open("austin.osm", "r")

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
street_types = defaultdict(set)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Bend", "Chase", "Circle", "Cove", "Crossing", "Hill",
            "Hollow", "Loop", "Park", "Pass", "Overlook", "Path", "Plaza", "Point", "Ridge", "Row",
            "Run", "Terrace", "Walk", "Way", "Trace", "View", "Vista"]

def audit_street_type(street_types, street_name):
	m = street_type_re.search(street_name)
	if m:
		street_type = m.group()
		if street_type not in expected:
			street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit():
    """ Parses through document and audits the tags for street names """
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    pprint.pprint(dict(street_types))

if __name__ == '__main__':
	audit()

{'300': {'West 5th Street #300'},
 '35': {'N Interstate Highway 35'},
 'Ave': {'South Congress Ave'},
 'Blvd.': {'South Exposition Blvd.'},
 'Chavez': {'West Cesar Chavez'},
 'Expressway': {'South MoPac Expressway'},
 'Gonzales': {'Gonzales'},
 'Greenway': {'Greenway'},
 'Jr': {'Robert Martinez Jr'},
 'Lanes': {'Green Lanes'},
 'River': {'Red River'},
 'Speedway': {'Speedway'},
 'St': {'W 10th St'}}


In [41]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "austin.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Bend", "Chase", "Circle", "Cove", "Crossing", "Hill",
            "Hollow", "Loop", "Park", "Pass", "Overlook", "Path", "Plaza", "Point", "Ridge", "Row",
            "Run", "Terrace", "Walk", "Way", "Trace", "View", "Vista"]

MAPPING = { "St": "Street",
            "St.": "Street",
            "Aceneu": "Avenue",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Bouevard": "Boulevard",
            "Cir": "Circle",
            "Ct": "Court",
            "Dr": "Drive",
            "Dr.": "Drive",
            "HWY": "Highway",
            "Ln": "Lane",
            "Pkwy": "Parkway",
            "Pl": "Plaza",
            "RD": "Road",
            "Rd": "Road",
            "Rd.": "Road",
            "Tr": "Trace",
            "Ter": "Terrace",
            "avenue": "Avenue",
            "blvd": "Boulevard",
            "road": "Road",
            "street": "Street",
            "court": "Court",
            "cove": "Cove",
            "lane": "Lane",
            "pass": "Pass"
            }           

def audit_street_type(street_types, street_name):
    """ 
        Creates a list of streets not in expected list 
        
        Args:
            street_types: Set containing unexpected stree types
    """
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
	return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    """ 
        Parses through document and audits the street tags 
        Args:
            osmfile: Data from OpenStreetMap
        Returns:
            street_types: Set containing unexpected street names
    """
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    elem.clear()
    return street_types

def update_street_name(name,MAPPING):
    """ 
        Replaces unexpected street names with better names 
        Args:
            name: An unexpected street name
            mapping: Dictionary of expected street names
        Returns:
            name: The updated street name
    """
    words = name.split()
    for w in range(len(words)):
    	if words[w] in MAPPING:
    		words[w] = MAPPING[words[w]]
    name = " ".join(words)
    return name

def final():
    st_types = audit(OSMFILE)
    pprint.pprint(dict(st_types))

    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_street_name(name, MAPPING)
            print (name, "=>", better_name)

if __name__ == '__main__':
    final()

{'120': {'Building B Suite 120'},
 '1400': {'Lavaca Suite 1400'},
 '300': {'West 5th Street #300'},
 '35': {'North Interstate Highway 35', 'N Interstate Highway 35'},
 '5.700': {'East 23rd Street, North End Zone, Suite 5.700'},
 'Ave': {'South Congress Ave'},
 'Blvd': {'S Lamar Blvd'},
 'Blvd.': {'South Exposition Blvd.', 'South Lamar Blvd.'},
 'Chavez': {'West Cesar Chavez'},
 'D5000': {'Speedway Stop D5000'},
 'Dr': {'Walter Seaholm Dr'},
 'Expressway': {'South MoPac Expressway'},
 'Gonzales': {'Gonzales'},
 'Grande': {'Rio Grande'},
 'Greenway': {'Greenway'},
 'IH-35': {'N IH-35'},
 'Jacinto': {'San Jacinto'},
 'Jr': {'Robert Martinez Jr'},
 'Lanes': {'Green Lanes'},
 'Quarry': {'Quarry'},
 'Rd': {'N Interstate 35 Frontage Rd'},
 'River': {'Red River'},
 'Riverside': {'W Riverside'},
 'Speedway': {'Speedway'},
 'St': {'E 6th St',
        'Rio Grande St',
        'S 1st St',
        'W 10th St',
        'W 6th St',
        'W Annie St',
        'West Lynn St'},
 'St.': {'E 38th 1/2 S

In [123]:
## Cleaning Street Names
#The first part of the data that I cleaned was the abbreviated street names. 
#To start I used the above code 
#to pares through the way tags and return street names that were uncommon, 
#according to a predefined list that I created. 
#I used to iterate over the tags and map the more explicit name to each street name as below.


In [73]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "austin.osm"

def audit_zipcode(zipcode_types, zipcode_name):
	""" 
		Checks if valid postal code. Adds invalid postal
		codes to set.
		
		Args:
			zipcode_types: Set containing invalid postal codes
			zipcode_name: Zipcode for the element
	"""
	if not (re.match(r'^(78|73)\d{3}$', zipcode_name)):
		zipcode_types[firstTwo].add(zipcode_name)

def is_zipcode(elem):
	""" 
		Checks the attribute for postal code 
		
		Args:
			elem: The element from the data point
		Returns:
			True if valid postal code. False otherwise.
	"""
	return (elem.attrib['k'] == "addr:postcode")

def audit(osmfile):
	""" 
		Parses the document and audits the postal codes 
		
		Args:
			osmfile: OpenStreetMap data
		Returns:
			zipcode_types: Set containing invalid zipcodes
	"""
	osm_file = open(osmfile, "r")
	zipcode_types = defaultdict(set)
	for event, elem in ET.iterparse(osm_file, events=("start",)):
		if elem.tag == "node" or elem.tag == "way":
			for tag in elem.iter("tag"):
				if is_zipcode(tag):
					audit_zipcode(zipcode_types, tag.attrib['v'])
	elem.clear()
	return zipcode_types

def update_zipcode(zip):
	""" 
		Update invalid postal codes
		Args:
			zip: A zipcode from the zipcodes set
		Returns:
			better_zip: A formatted zipcode
	"""
	char = re.findall('[a-zA-z]*', zip)
	if char:
		char = char[0]
	char = char.strip()
	if char == "TX" or "tx":
		better_zip = re.findall(r'\d+', zip)
		if better_zip:
			if len(better_zip) == 2:
				return better_zip[0]
			else:
				return better_zip
	else:
		better_zip = re.findall(r'\d+', zip)
		if better_zip:
			if len(better_zip) == 2:
				return better_zip[0]
			else:
				return better_zip

def final():
    zipcodes = audit(OSMFILE)
    pprint.pprint(dict(zipcodes))

    for zipcode, ways in zipcodes.items():
        for name in ways:
            better_name = update_zipcode(name)
            print (name, "=>", better_name)

if __name__ == '__main__':
    final()


{}


In [122]:
## Problems Encountered
#Some of the problems that I noticed:
#* The format for street names was not ubiquitous with some street names being abbreviated and others not 
#having the first letter capitalized.

#* Inconsistent postal codes. Some of the codes were formatted with like `78665`, `78724-1199`, and `TX 78613`.

In [ ]:
## Preparing Data for SQL
#In order to prepare the data for SQL, parsed through the XML data and converted 
#it into tabular form that I then used to create .csv files. Once in csv format it
#was easy to upload the data to a MySQL database. The code for this process can be found

In [79]:
# create nodes table
node_cols = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
osm_file = "austin.osm"
import collections
import pandas as pd
collections.defaultdict(int)
def nodes_table ():
    austin = open(osm_file,"r")
    nodes = collections.defaultdict(list)
    for _,elem in ET.iterparse(austin, events = ("start",)):
        if elem.tag == "node":
            for col in node_cols:
                try:    
                    nodes[col].append(elem.attrib[col].encode('utf-8'))
                except:
                    nodes[col].append(float('nan'))
    return pd.DataFrame(nodes).fillna(method='ffill')
nodes_table().to_csv('nodes.csv', index=True, header=False)
nodes = nodes_table()
nodes.head(10)

,changeset,id,lat,lon,timestamp,uid,user,version
0,b'27935577',b'152367979',b'30.2765560',b'-97.7023277',b'2015-01-05T15:39:51Z',b'1916849',b'nubs',b'3'
1,b'27935577',b'152367980',b'30.2773145',b'-97.7024904',b'2015-01-05T15:39:51Z',b'1916849',b'nubs',b'3'
2,b'3221556',b'152367981',b'30.2780860',b'-97.7026380',b'2009-11-26T16:49:07Z',b'147510',b'woodpeck_fixbot',b'2'
3,b'59235871',b'152367983',b'30.2788262',b'-97.7027940',b'2018-05-24T09:28:59Z',b'6294893',b'teodorab_telenav',b'3'
4,b'3221556',b'152367985',b'30.2796220',b'-97.7029760',b'2009-11-26T16:49:07Z',b'147510',b'woodpeck_fixbot',b'2'
5,b'2185759',b'152367986',b'30.2799630',b'-97.7030600',b'2009-08-18T07:24:15Z',b'147510',b'woodpeck_fixbot',b'2'
6,b'3221556',b'152367988',b'30.2803580',b'-97.7032860',b'2009-11-26T16:49:07Z',b'147510',b'woodpeck_fixbot',b'2'
7,b'3221556',b'152367990',b'30.2810630',b'-97.7037650',b'2009-11-26T16:49:07Z',b'147510',b'woodpeck_fixbot',b'2'
8,b'2185759',b'152367992',b'30.2816920',b'-97.7042510',b'2009-08-18T07:24:15Z',b'147510',b'woodpeck_fixbot',b'2'
9,b'28833447',b'152368021',b'30.2671816',b'-97.7323270',b'2015-02-14T01:33:25Z',b'119881',b'clay_c',b'4'


In [82]:
#creat ways table
way_cols = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
def ways_table():
    pittsburgh = open(osm_file,"r")
    ways= defaultdict(list)
    for _, elem in ET.iterparse(pittsburgh, events=("start",)):
        if elem.tag == "way":
            for col in way_cols:
                try:
                    ways[col].append(elem.attrib[col].encode('utf-8'))
                except:
                    ways[col].append(float('nan'))
    return pd.DataFrame(ways).fillna(method='ffill')
ways_table().to_csv('ways.csv', index=True, header=False)
ways = ways_table()
ways.head(10)

,changeset,id,timestamp,uid,user,version
0,b'10551674',b'15373463',b'2012-01-31T20:27:50Z',b'92286',b'Paul Johnson',b'3'
1,b'10551674',b'15373468',b'2012-01-31T19:15:48Z',b'92286',b'Paul Johnson',b'3'
2,b'41418034',b'15373490',b'2016-08-12T20:16:13Z',b'4124521',b'Money G',b'10'
3,b'26835242',b'15373607',b'2014-11-17T02:02:33Z',b'703517',b'Iowa Kid',b'19'
4,b'27321661',b'15373614',b'2014-12-07T21:43:42Z',b'703517',b'Iowa Kid',b'6'
5,b'27623048',b'15373728',b'2014-12-22T04:38:45Z',b'703517',b'Iowa Kid',b'6'
6,b'41980333',b'15373800',b'2016-09-07T14:13:36Z',b'1110270',b'afdreher',b'13'
7,b'53829051',b'15373880',b'2017-11-16T05:24:58Z',b'5680733',b'venkanna37',b'19'
8,b'29408663',b'15373883',b'2015-03-11T16:00:35Z',b'37392',b'25or6to4',b'5'
9,b'41376722',b'15373891',b'2016-08-10T21:56:08Z',b'2508151',b'ridixcr',b'11'


In [ ]:
#The csv files are cleaned using EXCEL to reach the below format

In [84]:
df = pd.read_csv('nodes.csv')
df.head(10)

,Unnamed: 0,Changeset,id,lat,long,timestamp,uid,user,version
0,0,27935577,152367979,30.276556,-97.702328,2015-01-05T15:39:51Z,1916849,nubs,3
1,1,27935577,152367980,30.277315,-97.702490,2015-01-05T15:39:51Z,1916849,nubs,3
2,2,3221556,152367981,30.278086,-97.702638,2009-11-26T16:49:07Z,147510,woodpeck_fixbot,2
3,3,59235871,152367983,30.278826,-97.702794,2018-05-24T09:28:59Z,6294893,teodorab_telenav,3
4,4,3221556,152367985,30.279622,-97.702976,2009-11-26T16:49:07Z,147510,woodpeck_fixbot,2
5,5,2185759,152367986,30.279963,-97.703060,2009-08-18T07:24:15Z,147510,woodpeck_fixbot,2
6,6,3221556,152367988,30.280358,-97.703286,2009-11-26T16:49:07Z,147510,woodpeck_fixbot,2
7,7,3221556,152367990,30.281063,-97.703765,2009-11-26T16:49:07Z,147510,woodpeck_fixbot,2
8,8,2185759,152367992,30.281692,-97.704251,2009-08-18T07:24:15Z,147510,woodpeck_fixbot,2
9,9,28833447,152368021,30.267182,-97.732327,2015-02-14T01:33:25Z,119881,clay_c,4


In [85]:
df = pd.read_csv('ways.csv')
df.head(10)

,Unnamed: 0,Changeset,id,tiemstamp,uid,user,version
0,0,10551674,15373463,2012-01-31T20:27:50Z,92286,Paul Johnson,3
1,1,10551674,15373468,2012-01-31T19:15:48Z,92286,Paul Johnson,3
2,2,41418034,15373490,2016-08-12T20:16:13Z,4124521,Money G,10
3,3,26835242,15373607,2014-11-17T02:02:33Z,703517,Iowa Kid,19
4,4,27321661,15373614,2014-12-07T21:43:42Z,703517,Iowa Kid,6
5,5,27623048,15373728,2014-12-22T04:38:45Z,703517,Iowa Kid,6
6,6,41980333,15373800,2016-09-07T14:13:36Z,1110270,afdreher,13
7,7,53829051,15373880,2017-11-16T05:24:58Z,5680733,venkanna37,19
8,8,29408663,15373883,2015-03-11T16:00:35Z,37392,25or6to4,5
9,9,41376722,15373891,2016-08-10T21:56:08Z,2508151,ridixcr,11


In [ ]:
#I then used SQL queries to obtain statistical information from the data. 
#I started out looking at size of the tables to get an idea of the amount of data included and
#to make sure that the files were uploaded. 


In [ ]:
CREATE TABLE Nodes ( 
    no_id DECIMAL(6,4),
    Changeset DECIMAL(6,4),
   	id DECIMAL(6,4),
   	lat DECIMAL(6,4),
   	longitude DECIMAL(6,4),
   	timestamps varchar(255),
   	uid DECIMAL(6,4),
   	users varchar(255),
   	version DECIMAL(6,4)
);
SHOW VARIABLES LIKE "secure_file_priv";
LOAD DATA LOCAL INFILE "/Users/amishi/Downloads/nodes.csv"
INTO TABLE Nodes
COLUMNS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
ESCAPED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

CREATE TABLE Ways ( 
	no_id DECIMAL(6,4),
	Changeset DECIMAL(6,4),
	id DECIMAL(6,4),
	tiemstamps varchar(255),
	uid DECIMAL(6,4),
	users varchar(255),
	version DECIMAL(6,4)
);

SHOW VARIABLES LIKE "secure_file_priv";
LOAD DATA LOCAL INFILE "/Users/amishi/Downloads/ways.csv"
INTO TABLE ways
COLUMNS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
ESCAPED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;


    
SELECT COUNT(*) FROM Nodes;

In [ ]:
+----------+
| COUNT(*) |
+----------+
|   340453 |
+----------+
1 row in set (0.10 sec)

In [ ]:
SELECT COUNT(*) FROM ways;
+----------+
| COUNT(*) |
+----------+
|    39979 |
+----------+

In [ ]:
SELECT 
    table_name AS `Table`, 
    round(((data_length + index_length) / 1024 / 1024), 2) `Size in MB` 
FROM information_schema.TABLES 
WHERE table_schema = "example"
    AND table_name = "Nodes";
SELECT 
    table_name AS `Table`, 
    round(((data_length + index_length) / 1024 / 1024), 2) `Size in MB` 
FROM information_schema.TABLES 
WHERE table_schema = "example"
    AND table_name = "ways";
    

In [ ]:
+-------+------------+
| Table | Size in MB |
+-------+------------+
| nodes |      31.56 |
+-------+------------+
1 row in set (0.00 sec)


In [ ]:
+-------+------------+
| Table | Size in MB |
+-------+------------+
| ways  |       3.52 |
+-------+------------+
1 row in set (0.00 sec)

In [ ]:
SELECT count(*) FROM   
     (SELECT uid,users FROM nodes
      UNION ALL
      SELECT uid,users FROM ways)    
     nodes group by uid order by count(*) Desc limit 30;

In [ ]:
30432

In [ ]:
Number of unique users:

SELECT COUNT(DISTINCT(users.uid))          
FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) users;

In [ ]:
+----------------------------+
| COUNT(DISTINCT(users.uid)) |
+----------------------------+
|                       850  |
+----------------------------+

In [ ]:
Top 10 zipcodes:

SELECT tags.value, COUNT(*) as count 
FROM (SELECT * FROM node_tags 
      UNION ALL 
      SELECT * FROM way_tags) tags
WHERE tags.key='postcode'
GROUP BY tags.value
ORDER BY count DESC
LIMIT 10;

In [ ]:
+-------+-------+
| value | count |
+-------+-------+
| 78645 | 10882 |
| 78734 |  5605 |
| 78653 |  3542 |
| 78660 |  3501 |
| 78669 |  3189 |
| 78641 |  2862 |
| 78704 |  2485 |
| 78746 |  2445 |
| 78759 |  2089 |
| 78738 |  1938 |
+-------+-------+

In [ ]:
Top 10 cities:

SELECT tags.value as count 
FROM (SELECT * FROM node_tags UNION ALL 
      SELECT * FROM way_tags) tags
WHERE tags.key LIKE '%city'
GROUP BY tags.value
ORDER BY count DESC
LIMIT 10;

In [ ]:
+-----------------+
| value           |  
+-----------------+
| Austin          | 
| Round Rock      |
| Kyle            | 
| Austin, TX      | 
| Cedar Park      | 
| Leander         | 
| Buda            |
| Pflugerville    | 
| Georgetown      | 
| West Lake Hills | 
+-----------------+

In [ ]:
Number of cafes, hotels, pubs, and restaurants:
    
SELECT value
FROM (SELECT * from node_tags as T UNION ALL 
      SELECT * from way_tags as Z) as Q
WHERE (value = 'restaurant' OR value = 'hotel' OR  
       value = 'pub' OR value = 'cafe')
GROUP BY value;

In [ ]:
+------------+----------+
| value      | COUNT(*) |
+------------+----------+
| cafe       |       85 |
| hotel      |      124 |
| pub        |       32 |
| restaurant |      425 |
+------------+----------+

In [116]:
# References
#This project was created in conjunction with the Data Analytics Nanodegree through Udacity
#Beaulieu, Alan. Learning sql. " O'Reilly Media, Inc.", 2009.
#McKinney, Wes. Python for data analysis: Data wrangling with Pandas, NumPy, and IPython. 